### Import Required Libraries and Set Up Environment Variables

In [3]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [4]:
# Set environment variables from the .env in the local environment
load_dotenv("../Starter_Code/M6_Starter_Code/.env")

#setting the api keys to variables
nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")
#test prints to make sure everything is working as intended
#print(nyt_api_key)
#print(tmdb_api_key)

### Access the New York Times API

In [5]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
base_query_url=f"{url}api-key={nyt_api_key}&fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}"
base_query_url

'https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=6gZp08FtAh3U8Bjh7UsgWa08aYnhWpjm&fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531'

In [6]:
# Create an empty list to store the reviews
reviews_list=[]

# loop through pages 0-19
for page in range(0,20):
    # create query with a page number
    query_url=f"{base_query_url}&page={str(page)}"
    # API results show 10 articles at a time
    print(query_url)
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(query_url)
    print(response)
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    if response.status_code == 200:
        try:
            articles=response.json()
            print(articles)
            if articles["response"]["docs"]:
                
                for article in articles["response"]["docs"]:
                    # loop through the reviews["response"]["docs"] and append each review to the list
                    reviews_list.append(article)
                    # Print the page that was just retrieved
                    print(f"Page {page} retrieved.")
            else:
                    # Print the page number that had no results then break from the loop
                print(f"Page {page} has no results.")
                break
        except Exception as e:
            print(f"JSON error: {e}")
            
    else:    
        print(f"error on page {page}: {response.status_code}")


https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=6gZp08FtAh3U8Bjh7UsgWa08aYnhWpjm&fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531&page=0
<Response [200]>
{'status': 'OK', 'copyright': 'Copyright (c) 2024 The New York Times Company. All Rights Reserved.', 'response': {'docs': [{'web_url': 'https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html', 'snippet': 'A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.', 'source': 'The New York Times', 'headline': {'main': '‘The Attachment Diaries’ Review: Love, Sick', 'kicker': None, 'content_kicker': None, 'print_headline': 'The Attachment Diaries', 'name': None, 'seo': None, 'sub': None}, 'keywords': [{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': '

In [7]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
articles5=articles["response"]["docs"][:5]
print(json.dumps(articles5, indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2019/02/12/movies/sorry-angel-review.html",
        "snippet": "Set in early 1990s Paris, Christophe Honor\u00e9\u2019s new film presents us with a middle-aged, H.I.V.-stricken writer falling, wearily, for a bookish 20-something.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018Sorry Angel\u2019 Review: A Love Affair Two Men Start, Knowing the End",
            "kicker": "Critic\u2019s Pick",
            "content_kicker": null,
            "print_headline": "Attraction: That\u2019s the Easy Part",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "Sorry Angel (Movie)",
                "rank": 2,
         

In [46]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
from pandas import json_normalize
reviews_df = json_normalize(reviews_list)
reviews_df.head(2)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [47]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

#defining a function that will extract the title from the headline.main col
def extract_func(headline):
    #checking that the unicode characters are found in the headline.main col
    if '\u2018' in headline and '\u2019' in headline:
        """ Extracts the title from the headline text between 
        unicode characters \u2018 and \u2019.
    Variables:
        a: finds the first character after \u2018 
        b: finds the last character which is right before \u2019
        title (str): pulls the information between a and b into a string and
            adds "Review" to the end. 
    Args: 
        headline (str): The headline text.
        
    Returns:
        title (str): The extracted title or the original headline 
        if the characters are not found.
    """
        

        a = headline.find('\u2018')+1       
        b = headline.rfind('\u2019')
        title = headline[a:b]
        title += " Review"
    # if there are no unicode characters, it will return the 
    # headline.main as the title    
    else:
        title = headline
        title += " Review"
    #returns title for use in creating a new column
    return title

#adding the column title to the dataframe and applying the extract function to
#the headline.main col
reviews_df["title"]=reviews_df["headline.main"].apply(extract_func)
reviews_df.head(5)


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries Review
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It? Review
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever Review
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love Review
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children Review


In [48]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df["keywords"]=reviews_df["keywords"].apply(extract_keywords)
reviews_df.head(3)


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries Review
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It? Review
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever Review


In [49]:
# Create a list from the "title" column using to_list()
title_list=reviews_df["title"].tolist()
title_list
# These titles will be used in the query for The Movie Database


['The Attachment Diaries Review',
 'What’s Love Got to Do With It? Review',
 'You Can Live Forever Review',
 'A Tourist’s Guide to Love Review',
 'Other People’s Children Review',
 'One True Loves Review',
 'The Lost Weekend: A Love Story Review',
 'A Thousand and One Review',
 'Your Place or Mine Review',
 'Love in the Time of Fentanyl Review',
 'Pamela, a Love Story Review',
 'In From the Side Review',
 'After Love Review',
 'Alcarràs Review',
 'Nelly & Nadine Review',
 'Lady Chatterley’s Lover Review',
 'The Sound of Christmas Review',
 'The Inspection Review',
 'Bones and All Review',
 'My Policeman Review',
 'About Fate’ Review: Love the One You Review',
 'Waiting for Bojangles Review',
 'I Love My Dad Review',
 'A Love Song Review',
 'Alone Together Review',
 'Art of Love Review',
 'The Wheel Review',
 'Thor: Love and Thunder’ Review: A God Review',
 'Both Sides of the Blade Review',
 'Fire of Love Review',
 'Love & Gelato Review',
 'Stay Prayed Up Review',
 'Benediction’ Review:

### Access The Movie Database API

In [58]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key
print(url+tmdb_key_string)

https://api.themoviedb.org/3/search/movie?query=&api_key=28e2274d8bd40e0a2472788879c22701


In [63]:
# Create an empty list to store the results
tmbd_results_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter=0


# Loop through the titles
for t in title_list:
    # Check if we need to sleep before making a request
    if request_counter >0 and request_counter % 50 == 0:
        time.sleep(12)

    # Add 1 to the request counter
        request_counter+=1
    
    # Perform a "GET" request for The Movie Database
        search = f"{url}{t}{tmdb_key_string}"
        response_tmbd=requests.get(search)
        movie_data = response_tmbd.json()
        print(search)
        
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
        try:
            # Get movie id
            movie_id = response_tmbd["results"][0]["id"]
            # Make a request for a the full movie details
            movie_url = f'https://api.themoviedb.org/3/movie/{movie_id}/credits?{tmdb_key_string}'
            # Execute "GET" request with url
            movie_response=requests.get(movie_url)
            movie_info = movie_response.json()
            # Extract the genre names into a list
            genres= [genre["name"] for genre in movie_info["genres"]]

            # Extract the spoken_languages' English name into a list
            lang = [lang["english_name"] for lang in movie_info["spoken_languages"]]

            # Extract the production_countries' name into a list
            prod_country = [country["name"] for country in movie_info["production_countries"]]

            # Add the relevant data to a dictionary and
            # append it to the tmdb_movies_list list
            movie_dict={
                "title":movie_info["title"],
                "genres": genres,
                "spoken_languages": lang,
                "production_countries":prod_country
            }
            tmbd_results_list.append(movie_dict)
            # Print out the title that was found
            print(f"Found {t}")
        except:
            print(f"No results for {t}")
            print(movie_url)
    else:
        print(f"error:")

error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:
error:


In [12]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data


[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "genre": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "language": "es",
        "spoken_languages": [
            "Spanish"
        ],
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 0.708,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 4.0,
        "vote_count": 2,
        "production_countries": [
            "Argentina"
        ]
    },
    

In [13]:
# Convert the results to a DataFrame


,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",0.708,102,0,2021-10-07,4.000,2,[Argentina]
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[French, English]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,0,2023-03-24,7.643,21,[Canada]
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,"[Romance, Comedy, Adventure]",en,[English],https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",25.369,96,0,2023-04-21,6.389,122,[United States of America]
3,Other People's Children,Les Enfants des autres,0,[Drama],fr,[French],https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",16.246,104,0,2022-09-21,6.900,137,[France]
4,One True Loves,One True Loves,0,"[Romance, Drama, Comedy]",en,[English],,Emma and Jesse are living the perfect life tog...,28.872,100,37820,2023-04-07,6.681,36,"[Germany, United States of America]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,The Last Face,The Last Face,0,[Drama],en,"[English, Portuguese]",,"Miguel, a heroic Spanish doctor, puts himself ...",9.081,130,0,2017-01-11,5.617,184,[United States of America]
151,Lost in Paris,Paris pieds nus,0,[Comedy],fr,[French],,Fiona visits Paris for the first time to assis...,6.798,83,0,2017-01-14,6.051,108,"[Belgium, France]"
152,The Other Half,The Other Half,0,"[Drama, Comedy, Romance]",en,[English],,When English Mark surprises his American bride...,1.058,94,0,2006-05-15,6.708,12,[United Kingdom]
153,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,"[Romance, Drama, War]",en,"[English, Turkish]",https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",12.097,111,413844,2017-03-10,6.150,236,"[Turkey, United States of America]"


### Merge and Clean the Data for Export

In [14]:
# Merge the New York Times reviews and TMDB DataFrames on title


,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[French, English]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,"[Romance, Drama, Comedy]",en,[English],,Emma and Jesse are living the perfect life tog...,28.872,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,[Documentary],en,[],,May Pang lovingly recounts her life in rock & ...,2.368,97,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,[Drama],en,[English],https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,In Search of Fellini,In Search of Fellini,0,"[Drama, Adventure]",en,[English],,"Lucy, a small-town girl from Ohio, discovers t...",4.103,93,...,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,None,None,None,By Monica Castillo,"[{'firstname': 'Monica', 'middlename': None, '...",None
121,The Last Face,The Last Face,0,[Drama],en,"[English, Portuguese]",,"Miguel, a heroic Spanish doctor, puts himself ...",9.081,130,...,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
122,Lost in Paris,Paris pieds nus,0,[Comedy],fr,[French],,Fiona visits Paris for the first time to assis...,6.798,83,...,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None
123,The Other Half,The Other Half,0,"[Drama, Comedy, Romance]",en,[English],,When English Mark surprises his American bride...,1.058,94,...,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None


In [15]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"French, English",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,English,,Emma and Jesse are living the perfect life tog...,28.872,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,,,May Pang lovingly recounts her life in rock & ...,2.368,97,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,Drama,en,English,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [16]:
# Drop "byline.person" column


In [17]:
# Delete duplicate rows and reset index


,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"French, English",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,English,,Emma and Jesse are living the perfect life tog...,28.872,100,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,,,May Pang lovingly recounts her life in rock & ...,2.368,97,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,Drama,en,English,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None


In [18]:
# Export data to CSV without the index
